In [51]:
import pandas as pd
import numpy as np
from io import StringIO

In [147]:
ground_truth = pd.read_csv(StringIO("""id,class,predictionstring
1,Claim,1 2 3 4 5
1,Claim,6 7 8
1,Claim,21 22 23 24 25
1,Evidence,1 2 3 4 5
1,Evidence,6 7 8
1,Evidence,21 22 23 24 25"""))

predictions = pd.read_csv(StringIO("""id,class,predictionstring
1,Claim,1 2
1,Claim,6 7 8
2,Claim,12
1,Evidence,1 2 4 5
"""))

In [166]:
# let's first calculate true positives and false positives
from functools import reduce

def prediction_scores(group):
    prediction = set(group.iloc[0]["predictionstring_pred"].split())
    truth_sets = [set(g.split()) for i, g in group["predictionstring_truth"].iteritems() if type(g) == str]
    truth_scores = [(len(truth.intersection(prediction)) / max(len(prediction),len(truth))) >= 0.5 for truth in truth_sets ]
    correct_prediction = max(truth_scores,default=False)
    if correct_prediction:
        return "true positive"
    else:
        return "false positive"
    
merged = predictions.merge(ground_truth, how="left", on=["id","class"], suffixes=["_pred","_truth"])
grouped = merged.groupby(["id","class","predictionstring_pred"])
p = grouped.apply(prediction_scores).groupby("class").value_counts()

def truth_scores(group):
    truth = set(group.iloc[0]["predictionstring_truth"].split())
    prediction_sets = [set(g.split()) for i, g in group["predictionstring_pred"].iteritems() if type(g) == str]
    prediction_scores = [(len(truth.intersection(prediction)) / max(len(prediction),len(truth))) >= 0.5 for prediction in prediction_sets ]
    correct_prediction = max(prediction_scores,default=False)
    if correct_prediction:
        return "true positive"
    else:
        return "false negative"
    
merged = predictions.merge(ground_truth, how="right", on=["id","class"], suffixes=["_pred","_truth"])
grouped = merged.groupby(["id","class","predictionstring_truth"])
t = grouped.apply(truth_scores).groupby("class").value_counts()

counts = pd.DataFrame(p.append(t)).reset_index().drop_duplicates() # true positives should be in each one

counts
# now count the macro f3 score
def f3_score(group):
    """Given a series with the counts by category, calculate the f3"""
    return group.loc["true positive"]
    return group.get("true positive",0)  / (
        group.get("true positive",0) + 0.1 * group.get("false positive",0) + 0.9 * group.get("false negative",0))
#counts.groupby("class").apply(f3_score)
counts

,class,level_1,0
0,Claim,false positive,2
1,Claim,true positive,1
2,Evidence,true positive,1
3,Claim,false negative,2
5,Evidence,false negative,2
